In [395]:
% matplotlib tk

UsageError: Line magic function `%` not found.


In [389]:
import os
from os import listdir
from pathlib import *
from pathlib import Path
import cv2 as cv
import numpy as np
from PIL import Image
import matplotlib

matplotlib.use('TKAgg')
from matplotlib import pyplot as plt
from pixelmatch.contrib.PIL import pixelmatch as pm2

In [24]:
matplotlib.get_backend()

'TkAgg'

# Image lines segmentation

Find the edges of the image (i.e. the edges of the single lines) using the implemented Canny algorithm.


In [12]:
# Finding Contours
# Use a copy of the image e.g. edged.copy()
# since findContours alters the image

contours, hierarchy = cv.findContours(grey, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

In [14]:
# Contours visualization
# -1 = drawing all contours
cv.drawContours(prd, contours, -1, (0, 255, 0), 3)

array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0, 255,   0],
        [  0, 255,   0],
        [  0, 255,   0]]

In [400]:
def get_line(img, dr):
    img = cv.imread(img)
    grey = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    contours, hierarchy = cv.findContours(grey, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    line_num = len(contours)
    print("Number of line elements found: " + str(line_num))

    for x in range(len(contours)):
        #selecting line x of those detected
        bb = contours[x]
        #creating a stencil of the original image
        #1. preparing the black background image;
        stencil = np.zeros(img.shape).astype(img.dtype)
        #2. set into an array the actual contours of the 'line element';
        cont = [np.array(bb)]
        #3. fill the contours with white color (i.e. [255, 255, 255]);
        line = cv.fillPoly(stencil, cont, color=(255, 255, 255))

        #_._._Saving step
        dr = './' + str(dr) + '/'
        filename = dr + 'line' + str(x) + '.png'
        cv.imwrite(filename, line)

    return (line_num)

In [40]:
#GT lines segmentation
get_line('gt2.png', 'linegt')

Number of line elements found: 76


76

In [404]:
#PRED lines segmentation
get_line('fmb-cb-55-027v.png', 'lines')

Number of line elements found: 105


105

In [47]:
#dumb fun to count the num of .png file on a directory
def elements(dr):
    count = 0
    for x in os.listdir(dr):
        if x.endswith(".png"):
            count += 1
    return (count)

In [49]:
elements('./linegt/')

76

# Intersection over Union

In [388]:
#Defining a function to calculate IoU on the fly
def IoU(l1, l2):
    intersection = np.logical_and(l1, l2)
    union = np.logical_or(l1, l2)
    iou_score = np.sum(intersection) / np.sum(union)
    return (iou_score)

# Recall and Percision

Procedure: https://www.jeremyjordan.me/evaluating-image-segmentation-models/

In [393]:
def recall(gt, prd):

    #setting variables
    score = []
    TP = 0
    FN = 0
    threshold = .3
    dir1 = Path(gt).glob('*.png')
    dir2 = Path(prd).glob('*.png')

    #number of lines detected on the gt image
    gtn = sum(1 for _ in dir1)
    #number of lines detected on the pred image
    pn = sum(1 for _ in dir2)
    xtr_ln = abs(pn - gtn)
    print('gt lines: ' + str(gtn))
    print('prd lines: ' + str(pn))

    #Iterating through the gt lines found
    for idx, img in enumerate(os.listdir(gt)):
        if (img.endswith(".png")):
            image1 = cv.imread(gt + img)

            print(idx, idx, img)

            #Iterating through the prd lines found
            for idx2, img2 in enumerate(os.listdir(prd)):
                if (img2.endswith(".png")):
                    image2 = Image.open(prd + img2)

                    print(idx2, img2)

                    score.append(IoU(image1, image2))

            #Calculating FP and FN through predefined threshold
            if max(score) > threshold:
                TP += 1
            else:
                FN += 1
            score = []

    Rec = TP / (TP + FN)

    out = "Recall score: " + str(Rec)

    return (out)

In [407]:
def precision(gt, prd):

    #setting variables
    score = []
    TP = 0
    FP = 0
    threshold = .3
    dir1 = Path(gt).glob('*.png')
    dir2 = Path(prd).glob('*.png')

    #number of lines detected on the gt image
    gtn = sum(1 for _ in dir1)
    #number of lines detected on the pred image
    pn = sum(1 for _ in dir2)
    print('gt lines: ' + str(gtn))
    print('prd lines: ' + str(pn))

    #Iterating through the gt lines found
    for idx, img in enumerate(os.listdir(prd)):
        if (img.endswith(".png")):
            image1 = cv.imread(prd + img)

            print(idx, idx, img)

            #Iterating through the prd lines found
            for idx2, img2 in enumerate(os.listdir(gt)):
                if (img2.endswith(".png")):
                    image2 = Image.open(gt + img2)

                    print(idx2, img2)

                    score.append(IoU(image1, image2))

            #Calculating FP and FN through predefined threshold
            if max(score) > threshold:
                TP += 1
            else:
                FP += 1
            score = []

    Precision = TP / (TP + FP)

    out = "Precision score: " + str(Precision)

    return (out)

In [408]:
dir1 = '/Users/mappo/PycharmProjects/DIA/DIA-Project/linegt/'
dir2 = '/Users/mappo/PycharmProjects/DIA/DIA-Project/lines/'
precision(dir1, dir2)

gt lines: 76
prd lines: 105
0 0 line40.png
0 line40.png
1 line54.png
2 line68.png
3 line1.png
4 line0.png
5 line69.png
6 line55.png
7 line41.png
8 line57.png
9 line43.png
10 line2.png
11 line3.png
12 line42.png
13 line56.png
14 line52.png
15 line46.png
16 line7.png
18 line6.png
19 line47.png
20 line53.png
21 line45.png
22 line51.png
23 line4.png
24 line5.png
25 line50.png
26 line44.png
27 line23.png
28 line37.png
29 line36.png
30 line22.png
31 line34.png
32 line20.png
33 line21.png
34 line35.png
35 line19.png
36 line31.png
37 line25.png
38 line24.png
39 line30.png
40 line18.png
41 line26.png
42 line32.png
43 line33.png
44 line27.png
45 line16.png
46 line17.png
47 line15.png
48 line29.png
49 line28.png
50 line14.png
51 line38.png
52 line10.png
53 line11.png
54 line39.png
55 line13.png
56 line12.png
57 line61.png
58 line75.png
59 line49.png
60 line8.png
61 line9.png
62 line48.png
63 line74.png
64 line60.png
65 line62.png
66 line63.png
67 line73.png
68 line67.png
69 line66.png
70 line72.p

'Precision score: 0.18095238095238095'

In [406]:
dir1 = '/Users/mappo/PycharmProjects/DIA/DIA-Project/linegt/'
dir2 = '/Users/mappo/PycharmProjects/DIA/DIA-Project/lines/'
recall(dir1, dir2)

gt lines: 76
prd lines: 105
0 0 line40.png
0 line40.png
1 line54.png
2 line68.png
3 line83.png
4 line97.png
5 line1.png
6 line0.png
7 line96.png
8 line82.png
9 line69.png
10 line55.png
11 line41.png
12 line57.png
13 line43.png
14 line94.png
15 line80.png
16 line2.png
17 line3.png
18 line81.png
19 line95.png
20 line42.png
21 line56.png
22 line52.png
23 line46.png
24 line91.png
25 line85.png
26 line7.png
28 line6.png
29 line84.png
30 line90.png
31 line47.png
32 line53.png
33 line79.png
34 line45.png
35 line51.png
36 line86.png
37 line92.png
38 line4.png
39 line5.png
40 line93.png
41 line87.png
42 line50.png
43 line44.png
44 line78.png
45 line23.png
46 line37.png
47 line104.png
48 line36.png
49 line22.png
50 line34.png
51 line20.png
52 line21.png
53 line35.png
54 line19.png
55 line31.png
56 line25.png
57 line103.png
58 line102.png
59 line24.png
60 line30.png
61 line18.png
62 line26.png
63 line32.png
64 line100.png
65 line101.png
66 line33.png
67 line27.png
68 line16.png
69 line17.png
70 l

'Recall score: 0.25'

In [392]:
dir1 = '/Users/mappo/PycharmProjects/DIA/DIA-Project/linegt/'
dir2 = '/Users/mappo/PycharmProjects/DIA/DIA-Project/newline/'
recall(dir1, dir2)

gt lines: 76
prd lines: 81
0 0 line40.png
0 line40.png
1 line54.png
2 line68.png
3 line1.png
4 line0.png
5 line69.png
6 line55.png
7 line41.png
8 line57.png
9 line43.png
10 line80.png
11 line2.png
12 line3.png
13 line42.png
14 line56.png
15 line52.png
16 line46.png
17 line7.png
19 line6.png
20 line47.png
21 line53.png
22 line79.png
23 line45.png
24 line51.png
25 line4.png
26 line5.png
27 line50.png
28 line44.png
29 line78.png
30 line23.png
31 line37.png
32 line36.png
33 line22.png
34 line34.png
35 line20.png
36 line21.png
37 line35.png
38 line19.png
39 line31.png
40 line25.png
41 line24.png
42 line30.png
43 line18.png
44 line26.png
45 line32.png
46 line33.png
47 line27.png
48 line16.png
49 line17.png
50 line15.png
51 line29.png
52 line28.png
53 line14.png
54 line38.png
55 line10.png
56 line11.png
57 line39.png
58 line13.png
59 line12.png
60 line61.png
61 line75.png
62 line49.png
63 line8.png
64 line9.png
65 line48.png
66 line74.png
67 line60.png
68 line76.png
69 line62.png
70 line63.pn

'Recall score: 0.06578947368421052'